#Building a Chatbot with Pytorch

Sumber : [github](https://github.com/UsmanNiazi/Chatbot-with-Pytorch/blob/main/chatbot.ipynb)

###Import Libray

In [ ]:
import numpy as np
import random
import json
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

###Creating Custom Functions
- create custom functions so that it is easy for us to implement afterwards

In [ ]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

Nltk or natural language took kit is a really useful library that contains important classes that will be useful in any of your NLP task.

###Stemming
- There are different methods that we can use for stemming. Here we will use Porter Stemmer model form our NLTK Library.

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

###Bag of Words
- We will be splitting each word in the sentences and adding it to an array.
- We will loop over the each word in the all words array and the bog array corresponding to each word. If a word from the sentence is found in the all words array, 1 will be replaced at that index/position in bog array.

In [ ]:
def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

During the the process , we will also use *nltk.word_tokenize()* which will convert a single sentence string into a list of word.

---

**Note**: we will pass lower case words to the stemmer so that words like Good and good (capitalized) won’t be labelled as different words.

###Loading the Data & Cleaning it

In [ ]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

This will separate all the tags & words into their separate lists

###Cleaning & Preparing the data using our custom functions

In [ ]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

93 patterns
21 tags: ['deskripsi_Kabupaten Metro', 'deskripsi_Kabupaten_Lampung_Barat', 'deskripsi_Kabupaten_Lampung_Selatan', 'deskripsi_Kabupaten_Lampung_Tengah', 'deskripsi_Kabupaten_Lampung_Timur', 'deskripsi_Kabupaten_Lampung_Utara', 'deskripsi_Kabupaten_Mesuji', 'deskripsi_Kabupaten_Pesawaran', 'deskripsi_Kabupaten_Pesisir_Barat', 'deskripsi_Kabupaten_Pringsewu', 'deskripsi_Kabupaten_Tanggamus', 'deskripsi_Kabupaten_Tulang_Bawang', 'deskripsi_Kabupaten_Tulang_Bawang_Barat', 'deskripsi_Kabupaten_Way Kanan', 'deskripsi_Kabupaten_Way_Lima', 'deskripsi_Kota_Bandar_Lampung', 'deskripsi_Kota_Guning_Sugih', 'deskripsi_Kota_Liwa', 'deskripsi_Kota_Metro', 'goodbye', 'greeting']
47 unique stemmed words: ['afternoon', 'aku', 'bandar', 'barat', 'bawang', 'bro', 'gua', 'gunung', 'hai', 'hallo', 'halo', 'hei', 'hi', 'hy', 'kabupaten', 'kanan', 'kawan', 'ke', 'kota', 'lampung', 'lima', 'liwa', 'malam', 'mau', 'mesuji', 'metro', 'morn', 'pagi', 'pergi', 'pesawaran', 'pesisir', 'pringsewu', 'saya

###Creating Training Data

In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

##PyTorch Model

---
Here we will be making a class to implement our custom neural network. It will be a feed forward neural Network which will have 3 Linear Layers and we will be using activation function “ReLU” .
- **Note:** 
We have used the super() function to inherit the properties of its parent class. 
This is an Object Oriented Programming (OOP) concept.

###Creating our Model

In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)  # Dropout layer for regularization
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

Here we have inherited a class from NN.Module because we will be customizing the model & its layers

###Assigning the Dataset to the Model
- We will use some Magic functions, write our class.
- You can read online about __getitem__ and __getitem__ magic funtions.

In [ ]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

###Hyper Parameters
- Every Neural network has a set of hyper parameters that need to be set before use.
- Before Instantiating our Neural Net Class or Model that we wrote earlier, we will first define some hyper parameters which can be changed accordingly.

In [ ]:
# Hyper-parameters 
num_epochs = 2000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 16
output_size = len(tags)
print(input_size, output_size)

47 21


###Loss and Optimizer
- We will now Instantiate the model, loss and optimizer functions.
- Loss Function: Cross Entropy 
- Optimizer: Adam Optimizer

In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###Training the Model

In [ ]:
for epoch in range(num_epochs):
    print("Epoch ke-", epoch+1)
    # Calculate Accuracy
    n_correct = 0
    n_total = 0
    predicted_labels, ground_truth_labels = [], []

    for i, (words, labels) in enumerate(train_loader):
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Feedforward
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()
        n_total += labels.shape[0]

        # Appending
        predicted_labels.append(predicted.cpu().detach().numpy())
        ground_truth_labels.append(labels.cpu().detach().numpy())

    accuracy = 100 * n_correct / n_total

    # Print loss and accuracy
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.2f}, Acc: {accuracy:.2f}')


print(f'Final loss: {loss.item():.2f}')
print(f'Final accuracy: {accuracy:.2f}')

Streaming output truncated to the last 5000 lines.
Epoch ke- 53
Epoch ke- 54
Epoch ke- 55
Epoch ke- 56
Epoch ke- 57
Epoch ke- 58
Epoch ke- 59
Epoch ke- 60
Epoch ke- 61
Epoch ke- 62
Epoch ke- 63
Epoch ke- 64
Epoch ke- 65
Epoch ke- 66
Epoch ke- 67
Epoch ke- 68
Epoch ke- 69
Epoch ke- 70
Epoch ke- 71
Epoch ke- 72
Epoch ke- 73
Epoch ke- 74
Epoch ke- 75
Epoch ke- 76
Epoch ke- 77
Epoch ke- 78
Epoch ke- 79
Epoch ke- 80
Epoch ke- 81
Epoch ke- 82
Epoch ke- 83
Epoch ke- 84
Epoch ke- 85
Epoch ke- 86
Epoch ke- 87
Epoch ke- 88
Epoch ke- 89
Epoch ke- 90
Epoch ke- 91
Epoch ke- 92
Epoch ke- 93
Epoch ke- 94
Epoch ke- 95
Epoch ke- 96
Epoch ke- 97
Epoch ke- 98
Epoch ke- 99
Epoch ke- 100
Epoch [100/5000], Loss: 1.32, Acc: 73.12
Epoch ke- 101
Epoch ke- 102
Epoch ke- 103
Epoch ke- 104
Epoch ke- 105
Epoch ke- 106
Epoch ke- 107
Epoch ke- 108
Epoch ke- 109
Epoch ke- 110
Epoch ke- 111
Epoch ke- 112
Epoch ke- 113
Epoch ke- 114
Epoch ke- 115
Epoch ke- 116
Epoch ke- 117
Epoch ke- 118
Epoch ke- 119
Epoch ke- 120
Epo

In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

###Saving the Trained Model

In [ ]:
FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


###Loading our Saved Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=47, out_features=16, bias=True)
  (l2): Linear(in_features=16, out_features=16, bias=True)
  (l3): Linear(in_features=16, out_features=21, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

###Using the Chatbot

In [ ]:
bot_name = "LampungExBot"
print("Mau wisata kemanaa nihh?^^ (type 'bye' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "bye":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Mau wisata kemanaa nihh?^^ (type 'bye' to exit)


##Evaluasi